#### The first step is to import all of the libraries and source again

In [4]:
!conda install -c anaconda beautifulsoup4 --yes

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.0       |           py36_0         165 KB  anaconda
    ca-certificates-2020.1.1   |                0         132 KB  anaconda
    certifi-2020.4.5.1         |           py36_0         159 KB  anaconda
    openssl-1.1.1g             |       h7b6447c_0         3.8 MB  anaconda
    soupsieve-2.0              |             py_0          33 KB  anaconda
    ------------------------------------------------------------
                                           Total:         4.3 MB

The following NEW packages will be INSTALLED:

  beautifulsoup4     anaconda/linux-64::beautifulsoup4-4.9.0-py36_0
  soupsieve          anaconda/noarch::soupsieve-2.0-py_0

The following packages

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [18]:
!easy_install -m  lxml

Searching for lxml
Best match: lxml 4.5.0
Processing lxml-4.5.0-py3.6-linux-x86_64.egg

Using /home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/lxml-4.5.0-py3.6-linux-x86_64.egg
Because this distribution was installed --multi-version, before you can
import modules from this package in an application, you will need to
'import pkg_resources' and then use a 'require()' call similar to one of
these examples, in order to select the desired version:

    pkg_resources.require("lxml")  # latest installed version
    pkg_resources.require("lxml==4.5.0")  # this exact version
    pkg_resources.require("lxml>=4.5.0")  # this version or higher

Processing dependencies for lxml
Finished processing dependencies for lxml


In [5]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install html5lib

Note: you may need to restart the kernel to use updated packages.


In [4]:
url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050."
source = requests.get(url).text
Toronto_data = BeautifulSoup(source,)

#### Here, we create the dataframe we will work with

In [5]:
column_names = ['Postal code','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_names)
content = Toronto_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    toronto = toronto.append({'Postal code': postcode,'Borough': borough,'Neighborhood': neighborhood}, ignore_index=True)

#### Here we tidy the data by grouping

In [6]:
toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1
                                 
df = toronto.groupby(['Postal code','Borough'])['Neighborhood'].apply(', '.join).reset_index()

def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['Postal code', 'Borough'])
df2 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')
df2.rename(columns={'Postal code':'Postal Code'}, inplace=True)
df2.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### The next part is to load the geospatial data from the second source

In [7]:
geospatial_data = 'https://cocl.us/Geospatial_data'
!wget -q -O 'toronto_m.geospatial_data.csv' geospatial_data
Toronto_lat_lon_data = pd.read_csv(geospatial_data).set_index("Postal Code")
Toronto_lat_lon_data.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


#### Finally, we merge the two table using the on request, for which we use the unique postal codes as a center point

In [8]:
df = pd.merge(Toronto_lat_lon_data, df2, on='Postal Code')
df = df[['Postal Code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]
df.head(11)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [9]:
df.to_csv('Toronto_Part_2.csv',index=False)